## Input data preparation

#### This code is used to prepare various miscellaneous input data that are to be input into the eventual AAL calculations.
1. Copying rasters from HECRAS model files to a single folder and renaming it as required
2. Preparing Building data that will be fed into AAL calculations
3. If required, converting CRS and calculating coordinates

#### 1. Copying rasters from all RAS model files and copying it to a single folder and renaming it as required

In [22]:
import os
import shutil

In [23]:
# Directories
input_directory = r"U:\193709305\03_data\Probabilistic_Modeling\FloodManagerOutputs\TurkeyCreekResults"
output_directory = r"U:\193709305\03_data\Probabilistic_Modeling\FloodManagerOutputs\Turkey_wse_rasters_Slop"

os.makedirs(output_directory, exist_ok=True)

In [24]:
# Looping through first-level parent folders
for parent_folder in os.listdir(input_directory):
    parent_path = os.path.join(input_directory, parent_folder)

    if os.path.isdir(parent_path):
        # Looping through second level folders
        for folder in os.listdir(parent_path):
            folder_path = os.path.join(parent_path, folder)

            for folder1 in os.listdir(folder_path):
                folder1_path = os.path.join(folder_path, folder1)
                if os.path.isdir(folder1_path) and folder1.endswith(("q1", "q2", "q3", "q4")):     # Checking if the folder ends with q1, q2, q3, or q4
                    
                    for file in os.listdir(folder1_path):
                        if file.endswith("Slop.Turkey.Turkey_final_update.tif"):
                            old_file_path = os.path.join(folder1_path, file)
                            
                            # Renaming the filenames
                            new_filename = file.replace("_Slop.Turkey.Turkey_final_update", "")
                            print(new_filename)
                            new_file_path = os.path.join(output_directory, new_filename)
                            
                            shutil.copy2(old_file_path, new_file_path)                              # Copying file to Output directory
                            print(f"Copied and renamed: {file} → {new_filename}")

1000_L_q1.tif
Copied and renamed: 1000_L_q1_Slop.Turkey.Turkey_final_update.tif → 1000_L_q1.tif
1000_L_q2.tif
Copied and renamed: 1000_L_q2_Slop.Turkey.Turkey_final_update.tif → 1000_L_q2.tif
1000_L_q3.tif
Copied and renamed: 1000_L_q3_Slop.Turkey.Turkey_final_update.tif → 1000_L_q3.tif
1000_L_q4.tif
Copied and renamed: 1000_L_q4_Slop.Turkey.Turkey_final_update.tif → 1000_L_q4.tif
1000_U_q1.tif
Copied and renamed: 1000_U_q1_Slop.Turkey.Turkey_final_update.tif → 1000_U_q1.tif
1000_U_q2.tif
Copied and renamed: 1000_U_q2_Slop.Turkey.Turkey_final_update.tif → 1000_U_q2.tif
1000_U_q3.tif
Copied and renamed: 1000_U_q3_Slop.Turkey.Turkey_final_update.tif → 1000_U_q3.tif
1000_U_q4.tif
Copied and renamed: 1000_U_q4_Slop.Turkey.Turkey_final_update.tif → 1000_U_q4.tif
100_L_q1.tif
Copied and renamed: 100_L_q1_Slop.Turkey.Turkey_final_update.tif → 100_L_q1.tif
100_L_q2.tif
Copied and renamed: 100_L_q2_Slop.Turkey.Turkey_final_update.tif → 100_L_q2.tif
100_L_q3.tif
Copied and renamed: 100_L_q3_Slop

#### 2. Preparing building data files scuh that it is ready to be fed into the AAL calculations
- There were building footprints in the AIMS data that had ground elevation values as zero.
- So, this code extracts the ground elevation values from the Terrain raster file and associate them to the building footprints having zero ground elevation data

In [1]:
import os
import glob
import pathlib as pl

import numpy as np
import pandas as pd
import geopandas as gpd
from osgeo import gdal

In [16]:
def getTifData(tif_path: str):
    """Read a WSE raster and return the gdal objects"""
    src = gdal.Open(str(tif_path))
    rb = src.GetRasterBand(1)
    gt = src.GetGeoTransform()
    proj = src.GetProjection()
    return rb, gt, src

In [8]:
def query(x: float, y: float, gt: any, rb: any) -> float:
    """Queries one specific cell in the rasterband given an x, y in the 
    geotransform
    """
    px = int((x-gt[0]) / gt[1])   
    py = int((y-gt[3]) / gt[5])   
    return rb.ReadAsArray(px,py,1,1)[0][0]

In [9]:
def ElevationData(gdf_path, gt, rb):
    """Read structures, compute ground elevation values from the terrain file"""

    gdf = gpd.read_file(gdf_path) 
    
    for i, idx in enumerate(gdf.index):
        x = gdf.loc[idx, 'x_sp']
        y = gdf.loc[idx, 'y_sp']
        pixel_value = query(x, y, gt, rb)
        
        # Assign WSE value to the GeoDataFrame, if value is -9999 it assigns the same -9999 value
        gdf.at[idx, 'ground_elv'] = pixel_value

    return gdf

In [10]:
def ElevBuildingPts(tif_path, gdf_path):
    """Assigns ground elevation values to each building points"""
    rb, gt, src = getTifData(tif_path)
    elev_result = ElevationData(gdf_path, gt, rb)
    return elev_result

In [19]:
root_dir = pl.Path(os.getcwd())

tif_file = root_dir/'TurkeyBrush_raster/Turkey.Turkey_final_update.tif'
bld_file = root_dir/'misc_shps/TurkeyBrush_empty_empty_sa.shp'

elev_res = ElevBuildingPts(tif_file, bld_file)   # Calculating ground elevation

In [21]:
# Saving the final GeoDataFrame consisting of extracted elevation values
output_path_elev = root_dir/'misc_shps/TurkeyBrush_empty_empty_Elev.shp'
elev_res.to_file(output_path_elev, driver='ESRI Shapefile')

#### Assigning RES3 (3A to 3F) building types based on whether they have basements or not
- The building of occupancy type (RES3) are only categorized as RES3A, RES3B, ..., RES3F.
- It is not categorized if it has basement or not.
- So, the two lines of codes below this categorizes these RES3 occupancy types based on their availability of basements

In [28]:
def new_occtype(row, occtype_val, found_type_val):
    """Reads the  occupancy type and foundation type for RES3 occupancy types and reclassifies based on whether a structure has basement or not.
    Since RES3A to RES3F needed to be reclassified on their availability of the basement.
    """
    if row['occtype'] in occtype_val and row['found_type'] == found_type_val:
        return f"{row['occtype']}-{row['found_type']}"
    elif row['occtype'] in occtype_val and row['found_type'] != found_type_val:
        return f"{row['occtype']}-NB"
    else:
        return row['occtype']

In [29]:
# Reclassify RES3 occupancy type based on basement or not
root_dir = pl.Path(os.getcwd())
bldg_f = root_dir/'misc_shps/TurkeyBrush_final_BldAttrb.shp'

occtype_val = ["RES3A", "RES3B", "RES3C", "RES3D", "RES3E", "RES3F"]
found_type_val = "B"

res3_class = gpd.read_file(bldg_f)
res3_class['occ_new'] = res3_class.apply(new_occtype, args=(occtype_val, found_type_val), axis=1)

In [30]:
# Saving the reclassified RES3 categories shapefile
output_path_reclass = root_dir/'misc_shps/TurkeyBrush_finalfinal_BldAttrb.shp'
res3_class.to_file(output_path_reclass, driver = 'ESRI Shapefile')

#### Converting polygons shapefiles to point shapefiles
This is done for building polygons. The points are the centroid of the polygons.
- It is done because the building points are necessary to extract the WSE values from WSE rasters, to calculate AAL values.

In [7]:
from shapely.geometry import Point

root_dir = root_dir = pl.Path(os.getcwd()) 
bld_shp_path = root_dir.parent/'shps/polygon_shp/TurkeyBrush_finalfinal_BldAttrb.shp'
output_bld_shp_path = root_dir.parent/'shps/point_shp/TurkeyBrush_point_BldAttrb.shp'

In [10]:
bldg_gdf = gpd.read_file(bld_shp_path)
bldg_gdf['geometry'] = [Point(xy) for xy in zip(bldg_gdf['x_sp'], bldg_gdf['y_sp'])]   # Creating point geometry using centroid coordinates
bldg_gdf.to_file(output_bld_shp_path)

### Converting CRS and Calculating coordinates for the state plane projection

In [3]:
# To calculate coordinates for the state plane projection

shapefile_path = r"C:\Users\dneupane\Documents\Probabilistic\AAL_calc\Brush_100yr\shp_nsi\Bush_JoCo_nsi_proj.shp"
gdf = gpd.read_file(shapefile_path)

# Reproject to State Plane Kansas North (EPSG:26977)
#gdf = gdf.to_crs("EPSG:26977")

# Extract the new X and Y coordinates from the geometry
gdf['x_sp'] = gdf.geometry.x
gdf['y_sp'] = gdf.geometry.y

# Save the updated shapefile with new coordinates
output_path = r"C:\Users\dneupane\Documents\Probabilistic\AAL_calc\Brush_100yr\shp_nsi\Brush_nsi_state_proj.shp"
gdf.to_file(output_path)

print(f"Reprojected shapefile saved at: {output_path}")


Reprojected shapefile saved at: C:\Users\dneupane\Documents\Probabilistic\AAL_calc\Brush_100yr\shp_nsi\Brush_nsi_state_proj.shp
